<a href="https://colab.research.google.com/github/nicolaiberk/FrameCompetition/blob/main/code/embeddings/word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
## estimate word embeddings from newspaper data
## code adapted from https://github.com/damian0604/embeddingworkshop/blob/main/04exercise.ipynb
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import string
import re
import os
import pandas as pd
import csv
import sys
import ast
import time


# tqdm allows you to display progress bars in loops
from tqdm import tqdm
from datetime import datetime

import gensim

csv.field_size_limit(sys.maxsize)

# lets get more output
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# get full set of news articles
if not os.path.isfile('drive/MyDrive/frameComp/uniquesentences.txt') and not os.path.isdir('drive/MyDrive/frameComp/newspapers'):
    os.chdir('drive/MyDrive')
    os.system('mkdir frameComp')
    os.system('mkdir frameComp/newspapers')
    os.system('wget -O frameComp/newspapers/articles.zip https://www.dropbox.com/sh/2vfmbjrq6n1623z/AAA2JWpH5bLXdPe2IFSYqj1Za?dl=0')
    os.system('unzip frameComp/newspapers/articles.zip -d frameComp/newspapers')
    os.system('rm frameComp/newspapers/articles.zip')
    os.chdir('frameComp')

In [4]:
if not os.path.isfile('uniquesentences.txt'):
  # load all texts
  if 'artcls' not in locals():
    for filename in tqdm(os.listdir('newspapers')):
      if 'artcls' in locals():
        print(f'\nLoaded {artcls.shape[0]} articles')
        artcls = artcls.append(pd.read_csv('newspapers/'+filename))
      else:
        artcls = pd.read_csv('newspapers/'+filename)
    print(f'Loaded {artcls.shape[0]} articles, done.')

    artcls = artcls.reset_index()


  # keep only if string
  stringvar = [str == type(i) for i in artcls.text]
  artcls = artcls[stringvar]
  del(stringvar)

  print(artcls.text[0])

  9%|▉         | 1/11 [00:08<01:21,  8.17s/it]


Loaded 150648 articles


 18%|█▊        | 2/11 [00:26<02:04, 13.86s/it]


Loaded 369691 articles


 27%|██▋       | 3/11 [00:43<02:05, 15.64s/it]

...


Loaded 2063311 articles


 91%|█████████ | 10/11 [01:59<00:06,  6.28s/it]


Loaded 2162157 articles


100%|██████████| 11/11 [02:40<00:00, 14.56s/it]


Loaded 2474182 articles, done.
Zum ersten Mal seit dem Amoklauf von Newtown sind in den USA Befürworter und Gegner von schärferen Waffengesetzen vor den Senat getreten. Die frühere demokratische Abgeordnete Gabrielle Giffords, selbst Opfer einer Schusswaffen-Attacke, sagte an ihre ehemaligen Kollegen gerichtet: "Zu viele Kinder sterben. Zu viele Kinder. Wir müssen etwas unternehmen!" Giffords rief den Kongress zum Handeln auf. "Wir müssen etwas tun. Es wird schwer sein, aber jetzt ist die Zeit." Giffords war im Januar 2011 bei einem Besuch in ihrem Wahlkreis im Bundesstaat Arizona von einem jungen Mann aus nächster Nähe in den Kopf geschossen worden. Die Politikerin überlebte schwer verletzt. Bei der Attacke starben sechs Menschen, unter ihnen ein neunjähriges Mädchen. Giffords wurde von ihrem Ehemann Mark Kelly begleitet. Die Politikerin und der Ex-Astronaut hatten Anfang Januar die Initiative "Americans for Responsible Solutions" (Amerikaner für verantwortungsbewusste Lösungen) gegrü

In [5]:
if not os.path.isfile('uniquesentences.txt'):

  # subset
  artcls = artcls.text

  # cut into sentences
  print('\nCutting into sentences:')
  uniquesentences = set()
  trans = str.maketrans('', '', string.punctuation) # translation scheme for removing punctuation
  for review in tqdm(artcls):
    sentences = sent_tokenize(review) 
    for sentence in sentences:
      sent_trans = sentence.translate(trans).lower()
      if sent_trans not in uniquesentences:
        uniquesentences.add(sent_trans)

  del(artcls)


Cutting into sentences:


100%|██████████| 2214853/2214853 [32:03<00:00, 1151.32it/s]


In [6]:
if not os.path.isfile('uniquesentences.txt'):
  # extract 
  print(f"We now have {len(uniquesentences)} unique sentences.")

We now have 42300681 unique sentences.


In [7]:
if not os.path.isfile('uniquesentences.txt'):
  with open('uniquesentences.txt', 'w') as fo:
    writer = csv.writer(fo)
    for sentence in tqdm(uniquesentences):
      writer.writerow([sentence])

100%|██████████| 42300681/42300681 [04:00<00:00, 176153.23it/s]


In [8]:
if os.path.isfile('uniquesentences.txt'):
  with open('uniquesentences.txt') as fi:
    uniquesentences = fi.readlines()
  print(f"We now have {len(uniquesentences)} unique sentences.")

We now have 42301259 unique sentences.


In [9]:
tokenizedsentences = (sentence.split() for sentence in uniquesentences) # iterator for vocab definition

In [10]:
inp = "uniquesentences.txt"

In [11]:
print(f"Started setting up the model at {datetime.now()}")
model = gensim.models.Word2Vec(size=300, min_count=100, window = 5, workers = 4) # we want 300 dimensions and not overdo it with the features
model.build_vocab(tokenizedsentences)
print(f"Finished vocabulary definition at {datetime.now()}")

Started setting up the model at 2022-12-02 10:24:47.303264
Finished vocabulary definition at 2022-12-02 10:33:28.007436


In [12]:
del(uniquesentences)
del(tokenizedsentences)

In [13]:
from gensim.models.word2vec import LineSentence

print(f"Started training at {datetime.now()}")
model.train(LineSentence(inp), total_examples=model.corpus_count,  epochs=5)
print(f"Finished training at {datetime.now()}")

Started training at 2022-12-02 10:33:32.447392


Finished training at 2022-12-02 12:37:29.329102


In [15]:
print('Saving model:')
model.save("np_emb")
print('Model finished!')

Saving model:
Model finished!


In [16]:
# Store just the words + their trained embeddings.
model = gensim.models.Word2Vec.load("np_emb")
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")

## Assess model validity and bias

In [17]:
from gensim.models import KeyedVectors
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

In [18]:
wv.most_similar('flüchtling', topn=10)  # get other similar words

[('kriegsflüchtling', 0.7471936345100403),
 ('migrant', 0.7153430581092834),
 ('asylsuchender', 0.6903766393661499),
 ('afghane', 0.6758092641830444),
 ('syrer', 0.656206488609314),
 ('asylbewerber', 0.6547186374664307),
 ('häftling', 0.6372201442718506),
 ('eritreer', 0.6123487949371338),
 ('flüchtlingskind', 0.612052321434021),
 ('kurde', 0.6115912199020386)]

In [19]:
wv.most_similar('immigration', topn=10)

[('einwanderung', 0.8183763027191162),
 ('migration', 0.710647702217102),
 ('zuwanderung', 0.6837635040283203),
 ('arbeitsmigration', 0.6360291242599487),
 ('einwanderung“', 0.6120237708091736),
 ('migration“', 0.6078910827636719),
 ('masseneinwanderung', 0.5990856885910034),
 ('sekundärmigration', 0.573615550994873),
 ('massenmigration', 0.566524088382721),
 ('einwanderungspolitik', 0.5498449802398682)]

In [20]:
wv.most_similar(positive=["frau","könig"],negative=["mann"])

[('königin', 0.640566349029541),
 ('gemahlin', 0.6393531560897827),
 ('prinzessin', 0.6323640942573547),
 ('gattin', 0.6047043800354004),
 ('kaiserin', 0.5853005051612854),
 ('hofdame', 0.5741569995880127),
 ('mätresse', 0.5668904185295105),
 ('kronprinzessin', 0.5587282180786133),
 ('fürstin', 0.5533872246742249),
 ('regentin', 0.5430272817611694)]

In [21]:
wv.most_similar(positive=["frau","arzt"],negative=["mann"]) # surprisingly unbiased on gender

[('ärztin', 0.7031672596931458),
 ('therapeutin', 0.6784923076629639),
 ('hebamme', 0.6774612665176392),
 ('gynäkologin', 0.6630911231040955),
 ('patientin', 0.6559009552001953),
 ('frauenärztin', 0.643024742603302),
 ('zahnärztin', 0.6362940669059753),
 ('krankenschwester', 0.6322504878044128),
 ('kinderärztin', 0.6198192238807678),
 ('psychiaterin', 0.6130384206771851)]

In [22]:
wv.most_similar(positive=["sie","arzt"],negative=["er"]) # surprisingly unbiased on gender (at elast this example)

[('hausarzt', 0.6599127054214478),
 ('gynäkologen', 0.6542491912841797),
 ('frauenarzt', 0.6433839201927185),
 ('urologen', 0.6340921521186829),
 ('therapeuten', 0.62935471534729),
 ('ärzte', 0.6281211376190186),
 ('orthopäden', 0.6259489059448242),
 ('mediziner', 0.6251450777053833),
 ('radiologen', 0.6197144985198975),
 ('kinderarzt', 0.6165973544120789)]

In [23]:
wv.most_similar(positive=["paris","deutschland"],negative=["frankreich"])

[('berlin', 0.681095540523529),
 ('hamburg', 0.5988909006118774),
 ('london', 0.5880885124206543),
 ('wien', 0.5807464122772217),
 ('kopenhagen', 0.555401086807251),
 ('münchen', 0.5479743480682373),
 ('beirut', 0.5448683500289917),
 ('istanbul', 0.5434224605560303),
 ('stockholm', 0.542122483253479),
 ('bonn', 0.5394241809844971)]

In [24]:
wv.most_similar(positive=["paris","polen"],negative=["frankreich"])

[('warschau', 0.8180314302444458),
 ('prag', 0.750697910785675),
 ('budapest', 0.7054919004440308),
 ('krakau', 0.7040057182312012),
 ('breslau', 0.6801859736442566),
 ('bukarest', 0.6772236227989197),
 ('riga', 0.6729967594146729),
 ('vilnius', 0.6689883470535278),
 ('lodz', 0.6661868691444397),
 ('danzig', 0.6632505655288696)]

In [25]:
wv.most_similar(positive=["marokkaner","polizist"],negative=["deutscher"])

[('algerier', 0.6344766616821289),
 ('mann', 0.6289346218109131),
 ('afghane', 0.6286822557449341),
 ('angeklagte', 0.6121425628662109),
 ('tunesier', 0.6113818287849426),
 ('messerstecher', 0.6103597283363342),
 ('nigerianer', 0.6099405884742737),
 ('unfallfahrer', 0.6057000756263733),
 ('taxifahrer', 0.6055822968482971),
 ('messermann', 0.6040973663330078)]